# Snowflake SQL Alchemy

### Uploading the pdf metdata into snowflake using Snowflake-SQLAlchemy into our created db

https://colab.research.gohttps://colab.research.google.com/drive/1n-pYQ78izf7u9QwJozfmxJvIRYTV7gX2?usp=sharing

In [ ]:
pip install python-dotenv

In [ ]:
pip install bcrypt

In [ ]:
pip install snowflake-sqlalchemy

In [1]:
#!/usr/bin/env python
import warnings
warnings.filterwarnings("ignore")
from dotenv import load_dotenv
from sqlalchemy import create_engine
import bcrypt
import os
import csv
import re

In [2]:
import csv

def convert_to_url(column_data):
    # Constructing the URL based on the first column data
    trimmed_data = column_data[5:7]
    url = f"https://s3.console.aws.amazon.com/s3/object/dsd-tes-fpdr?region=us-east-1&bucketType=general&prefix=Grobid_RR_2024_{trimmed_data}_combined.txt"
    return url

# Open the CSV file for reading and writing
with open('metadata.csv', mode='r') as file:
    reader = csv.DictReader(file)
    fieldnames = reader.fieldnames + ['URL']
    
    # Create a new CSV file with an additional column for the URLs
    with open('metadata_new.csv', mode='w', newline='') as output_file:
        writer = csv.DictWriter(output_file, fieldnames=fieldnames)
        writer.writeheader()
        
        # Iterate over each row in the CSV file
        for row in reader:
            column_data = row['Filename']  # Assuming the first column contains the data for conversion
            url = convert_to_url(column_data)
            row['URL'] = url
            
            # Write the modified row to the new CSV file
            writer.writerow(row)

print("Conversion completed. Output saved to output.csv")



Conversion completed. Output saved to output.csv


In [9]:
create_database_query = "CREATE OR REPLACE DATABASE pdf_data;"
create_table_query = """CREATE OR REPLACE TABLE pdf_metadata (
        filename STRING,
        Idno STRING,
        Title STRING,
        Header STRING,
        Paragraph STRING,
        Application STRING,
        URL STRING
        );"""

In [10]:
create_stage = """CREATE STAGE PDF_STAGING DIRECTORY = ( ENABLE = true );"""
upload_to_stage = """PUT file://metadata_new.csv @pdf_data.public.pdf_staging;"""

In [11]:
create_warehouse = """CREATE OR REPLACE WAREHOUSE pdf WITH
   WAREHOUSE_SIZE='X-SMALL'
   AUTO_SUSPEND = 180
   AUTO_RESUME = TRUE
   INITIALLY_SUSPENDED=TRUE;
   """

In [12]:
copy_stage_to_table = """COPY INTO pdf_metadata
  FROM @pdf_data.public.pdf_staging
  FILE_FORMAT = (type = csv field_optionally_enclosed_by='"' skip_header = 1)
  PATTERN = 'metadata_new.csv.gz'
  ON_ERROR = 'skip_file';"""

In [13]:
load_dotenv()

u=os.getenv("SNOWFLAKE_USER")
p=os.getenv("SNOWFLAKE_PASS")
ai=os.getenv("SNOWFLAKE_ACCOUNTID")


engine = create_engine(
    'snowflake://{user}:{password}@{account_identifier}/'.format(
        user=u,
        password=p,
        account_identifier=ai,
    )
)



try:
    connection = engine.connect()
    results = connection.execute(create_database_query)
    results = connection.execute(create_table_query)
    results = connection.execute(create_warehouse)
    connection.execute("USE DATABASE pdf_data")
    results = connection.execute(create_stage)
    results = connection.execute(upload_to_stage)
    connection.execute("USE WAREHOUSE pdf")
    results = connection.execute(copy_stage_to_table)

finally:
    print("Done")
    connection.close()
    engine.dispose()

Done
